- Author: Ben Du- Date: 2020-09-01 16:31:49
- Title: Using Bucketing in Spark
- Slug: using-bucketing-in-spark
- Category: Computer Science
- Tags: Computer Science

In [3]:
import findspark
findspark.init("/opt/spark")

from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import StructType
spark = SparkSession.builder.appName("PySpark_Union") \
    .enableHiveSupport().getOrCreate()

Exception: Unable to find py4j, your SPARK_HOME may not be configured correctly

In [4]:
!ls /opt/